# Homework 6

## Problem 1

### (a)

Yes, if $\alpha$ is constant but sufficiently small, GD will converge to a minimizer of $f$.

### (b)

No, even if the $\alpha$ is suffiently small, SGM still has no guarantee to converge to the minimizer of $f$.

### (c)

Yes, there exists a $\alpha_k$ under certain conditions that makes SGM to converge to a minimizer of $f$.

## Problem 2

### 1

According to the given equation of hinge loss, we define the term $\left< \mathbf{w}, \mathbf{x}_i \right> + b$ in function $J(\mathbf{w}, b)$ as $t_i$, thus the given empirical risk can be rewritten as:

$$
\begin{alignat*}{2}
J(\mathbf{w}, b) &= \frac{1}{n} \sum_{i=1}^{n} (\text{max}\left\{ 0, 1 - y_i t_i \right\}) + \frac{\lambda}{2} \left\| \mathbf{w} \right\|^2 \\
&= \sum_{i=1}^{n} \frac{1}{n} (\text{max}\left\{ 0, 1 - y_i t_i \right\}) + \frac{\lambda}{2} \left\| \mathbf{w} \right\|^2 \\
&= \sum_{i=1}^{n} \left(\frac{1}{n} \text{max}\left\{ 0, 1 - y_i (\left< \mathbf{w}, \mathbf{x}_i \right> + b) \right\} + \frac{\lambda}{2n} \left\| \mathbf{w} \right\|^2 \right).
\end{alignat*}
$$

Therefore, $J_i (\mathbf{w}, b)$ can be interpreted as:
$$
J_i (\mathbf{w}, b) = \frac{1}{n} \text{max}\left\{ 0, 1 - y_i (\left< \mathbf{w}, \mathbf{x}_i \right> + b) \right\} + \frac{\lambda}{2n} \left\| \mathbf{w} \right\|^2.
$$

### 2

Firstly, we should further rewrite $J_i$ to be more readable

$$
J_i (\mathbf{w}, b) = \frac{1}{n} \text{max}\left\{ 0, 1 - y_i ( \mathbf{w}^\top \mathbf{x}_i + b) \right\} + \frac{\lambda}{2n} \left\| \mathbf{w} \right\|^2,
$$

subsequently, use $\mathbf{\theta}$ to simplifiy $J_i$

$$
\begin{alignat*}{2}
J_i (\mathbf{w}, b) &= \frac{1}{n} \text{max}\left\{ 0, 1 - y_i \mathbf{\theta}^\top \tilde{\mathbf{x}}_i \right\} + \frac{\lambda}{2n} \left\|
\begin{bmatrix}
0 & \\
  & \mathbf{I}
\end{bmatrix} \cdot \mathbf{\theta}
\right\|^2 \\
&= \frac{1}{n} \text{max}\left\{ 0, 1 - y_i \mathbf{\theta}^\top \tilde{\mathbf{x}}_i \right\} + \frac{\lambda}{2n} \mathbf{\theta}^\top
\begin{bmatrix}
0 & \\
  & \mathbf{I}
\end{bmatrix} \cdot \mathbf{\theta}
\\
\end{alignat*}
$$

where $\tilde{\mathbf{x}_i} = [1 , \mathbf{x_i}^\top]^\top$. We can define $\tilde{\mathbf{I}} = \begin{bmatrix}
0 & \\
  & \mathbf{I}
\end{bmatrix}$ for simplicity, thus $J_i$ can be further rewritten as

$$
J_i = \frac{1}{n} \text{max}\left\{ 0, 1 - y_i \mathbf{\theta}^\top \tilde{\mathbf{x}}_i \right\} + \frac{\lambda}{2n} \mathbf{\theta}^\top \tilde{\mathbf{I}} \mathbf{\theta}
\\
$$

With the equation above, its gradient w.r.t. $\mathbf{\theta}$ can be given as

$$
\mathbf{u}_i = 
\begin{cases}
\frac{1}{n}(- y_i \tilde{\mathbf{x}}_i + \lambda \tilde{\mathbf{I}}\mathbf{\theta}),~\text{if } y_i \mathbf{\theta}^\top \tilde{\mathbf{x}}_i \le 1 \\
\frac{\lambda}{n} \tilde{\mathbf{I}}\mathbf{\theta}, ~ \text{otherwise}
\end{cases}
$$